In [196]:
import os
import warnings
import findfile
import pandas as pd

In [197]:
import transformers
print(transformers.__version__)

4.28.0


In [198]:
import torch

In [199]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060 Ti'

In [200]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [201]:
from pyabsa import ABSAInstruction as absa_instruction

In [203]:
warnings.filterwarnings("ignore")

In [204]:
task_name = "multitask"
experiment_name = "instruction"
# model_checkpoint = "google/flan-t5-base"
model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"

In [205]:
print("Experiment Name: ", experiment_name)
model_out_path = "checkpoints"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

Experiment Name:  instruction
Model output path:  checkpoints\multitask\kevinscariaate_tk-instruct-base-def-pos-neg-neut-combined-instruction


In [216]:
id_train_file_path = "./ABSADatasets/datasets/acos_datasets/1000.Hotel/no_category"
id_test_file_path = "./ABSADatasets/datasets/acos_datasets/1000.Hotel/no_category"

In [215]:
import json

In [217]:
id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "train")

./ABSADatasets/datasets/acos_datasets/1000.Hotel/no_category\train.jsonl


In [218]:
id_tr_df

[{'text': 'Hotel has an amazing rooftop pool that is well maintained . Rooms are a good size and clean . However there is a noise problem sometimes when large groups come . Doors do bang quite loudly on closing .',
  'labels': [{'aspect': 'Doors',
    'opinion': 'loudly',
    'polarity': 'NEG',
    'category': None},
   {'aspect': 'large groups come',
    'opinion': 'noise problem',
    'polarity': 'NEG',
    'category': None},
   {'aspect': 'Rooms', 'opinion': 'good', 'polarity': 'POS', 'category': None},
   {'aspect': 'Rooms',
    'opinion': 'clean',
    'polarity': 'POS',
    'category': None},
   {'aspect': 'rooftop pool',
    'opinion': 'well maintained',
    'polarity': 'POS',
    'category': None},
   {'aspect': 'rooftop pool',
    'opinion': 'amazing',
    'polarity': 'POS',
    'category': None}]},
 {'text': 'For the four star hotel , the lobby design was very nice . The parking lot was far at the behind the hotel . My room decoration was very nice and lovely , just I find the

In [219]:
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

./ABSADatasets/datasets/acos_datasets/1000.Hotel/no_category\test.jsonl


In [220]:
id_tr_df = pd.DataFrame(id_tr_df)
id_tr_df

,text,labels
0,Hotel has an amazing rooftop pool that is well...,"[{'aspect': 'Doors', 'opinion': 'loudly', 'pol..."
1,"For the four star hotel , the lobby design was...","[{'aspect': 'others things', 'opinion': 'stand..."
2,I stayed for 2 nights in this hotel . it was g...,"[{'aspect': 'tourist attractions', 'opinion': ..."
3,"Nice hotel , good location , jerman beach just...","[{'aspect': 'money', 'opinion': 'value', 'pola..."
4,"Hotel is close to the airport , and the staffs...","[{'aspect': 'breakfast', 'opinion': 'standard'..."
...,...,...
1896,"Over all is good , but please maintenance your...","[{'aspect': 'internet', 'opinion': 'slowly', '..."
1897,We arrived late and stayed at Harris before de...,"[{'aspect': 'Staff', 'opinion': 'amazing', 'po..."
1898,Lovely hotel . Rooms are small . The location ...,"[{'aspect': 'hotel', 'opinion': 'Lovely', 'pol..."
1899,Overall is good . Food is good . Breakfast is ...,"[{'aspect': 'Food', 'opinion': 'good', 'polari..."


In [221]:
id_te_df = pd.DataFrame(id_te_df)
id_te_df

,text,labels
0,"Beautiful , comfortable and friendly hotel nea...","[{'aspect': 'money', 'opinion': 'Great value',..."
1,"Room veeeeery big , comfortable big bed , hot ...","[{'aspect': 'bed', 'opinion': 'comfortable', '..."
2,"room without power plug point , hair dryer , w...","[{'aspect': 'queen size bed', 'opinion': 'don ..."
3,The accomodation was great given the price 30 ...,"[{'aspect': 'Peti', 'opinion': 'close', 'polar..."
4,For the price this hotel excellent . Rooms are...,"[{'aspect': 'price', 'opinion': 'excellent', '..."
...,...,...
195,It ' s a new hotel located far from Kuta & Leg...,"[{'aspect': 'improvement', 'opinion': 'Need mo..."
196,"Arrive at Ozz Hotel , from the lobby I can see...","[{'aspect': 'room', 'opinion': 'reasonable', '..."
197,Swimming pool can not swim with open eyes salt...,"[{'aspect': 'Swimming pool', 'opinion': 'salt ..."
198,Fairly small room for hotel for its price rang...,"[{'aspect': 'room', 'opinion': 'small', 'polar..."


In [224]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [225]:
import json

# Load the updated train.jsonl file
file_path = 'ABSADatasets//datasets//acos_datasets//1000.Hotel//train.jsonl'

def load_and_check_data(file_path):
    data_entries = []
    with open(file_path, 'r') as file:
        for line in file:
            data_entries.append(json.loads(line))

    missing_keys_entries = []
    for entry in data_entries:
        for label in entry.get('labels', []):
            if 'aspect' not in label or 'polarity' not in label:
                missing_keys_entries.append(entry)
                break  # No need to check further if one label is problematic

    return missing_keys_entries

missing_keys_entries = load_and_check_data(file_path)

# Display entries with missing keys
missing_keys_entries

[]

In [226]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
# if loader.train_df_ood is not None:
#     loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [227]:
# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

[2024-06-03 19:47:15] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-06-03 19:47:15] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-06-03 19:47:15] (2.4.1.post1) Checkpoint:kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined is not found, you can raise an issue for requesting shares of checkpoints
[2024-06-03 19:47:15] (2.4.1.post1) No checkpoint found in Model Hub for task: kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined


In [228]:
# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 800/800 [00:00<00:00, 9810.89 examples/s]


In [140]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 200/200 [00:00<00:00, 7232.49 examples/s]


In [229]:
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 8,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    # 'fp16': True,
    "fp16": False,
}


In [241]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


 12%|█▎        | 1901/15208 [06:19<42:55,  5.17it/s]

{'eval_loss': 0.3331623077392578, 'eval_runtime': 10.1992, 'eval_samples_per_second': 78.438, 'eval_steps_per_second': 4.902, 'epoch': 1.0}


                                                       
 25%|██▌       | 3802/15208 [12:35<36:08,  5.26it/s]

{'eval_loss': 0.339480996131897, 'eval_runtime': 10.1863, 'eval_samples_per_second': 78.537, 'eval_steps_per_second': 4.909, 'epoch': 2.0}


...

{'eval_loss': 0.34088048338890076, 'eval_runtime': 10.1933, 'eval_samples_per_second': 78.483, 'eval_steps_per_second': 4.905, 'epoch': 7.0}


                                                       
100%|██████████| 15208/15208 [50:18<00:00,  5.19it/s]

{'eval_loss': 0.3478550612926483, 'eval_runtime': 10.1616, 'eval_samples_per_second': 78.728, 'eval_steps_per_second': 4.92, 'epoch': 8.0}


100%|██████████| 15208/15208 [50:21<00:00,  5.03it/s]


{'train_runtime': 3021.4151, 'train_samples_per_second': 20.134, 'train_steps_per_second': 5.033, 'train_loss': 0.1450681249948629, 'epoch': 8.0}


In [242]:
# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="train",
    batch_size=16,
)
id_tr_labels = [i.strip() for i in id_ds["train"]["labels"]]

Prediction from trainer


100%|██████████| 476/476 [07:07<00:00,  1.11it/s]


In [243]:
id_te_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="test",
    batch_size=16,
)
id_te_labels = [i.strip() for i in id_ds["test"]["labels"]]

Prediction from trainer


100%|██████████| 50/50 [00:44<00:00,  1.12it/s]


In [244]:
metrics = t5_exp.get_classic_metrics(id_tr_labels, id_tr_pred_labels)
print("----------------------- Classic Training Set Metrics -----------------------")
print(metrics)

time|location|staff|rooms|balconies|Breakfast|food|stay
staff|rooms|Breakfast|food
time:POS|location:POS|staff:POS|rooms:POS|balconies:POS|Breakfast:POS|food:POS|stay:POS
time:POS|location:POS|staff:POS|rooms:POS|balconies:POS|Breakfast:POS|food:POS|stay:POS
time:enjoyed|location:loved|staff:attentive|rooms:great|balconies:had|staff:friendly|Breakfast:included|food:delicious|stay:enjoyed
time:enjoyed|location:loved|staff:attentive|staff:friendly|rooms:great|balconies:had|Breakfast:included|food:delicious|stay:enjoyed
time:None|location:None|staff:None|rooms:None|balconies:None|staff:None|Breakfast:None|food:None|stay:None
time:None|location:None|staff:None|rooms:None|balconies:None|Breakfast:None|food:None|stay:None
teamofstaff|Activities|poolactivities|Service
teamofstaff|Agus|Ryan|Agung|poolactivities|Service|teamofstaff
teamofstaff:POS|Activities:POS|poolactivities:POS|Service:POS
teamofstaff:POS|Activities:POS|poolactivities:POS|Service:POS
teamofstaff:Worldclass|Activities:topqual

In [245]:
metrics = t5_exp.get_classic_metrics(id_te_labels, id_te_pred_labels)
print("----------------------- Classic Testing Set Metrics -----------------------")
print(metrics)

money|hotel|Rooms|Wifi|beach|mall
hotel|valueforthemoney|situation|mall|beach|Rooms|Wifi
money:POS|hotel:POS|Rooms:POS|Wifi:POS|beach:POS|mall:POS
money:POS|hotel:POS|Rooms:POS|Rooms:NEG|Wifi:POS|beach:POS|mall:POS
money:Greatvalue|hotel:friendly|hotel:comfortable|hotel:Beautiful|Rooms:comfortable|Rooms:clean|Rooms:nonoisy|Wifi:excellent|beach:nextto|mall:nextto
money:Greatvalue|hotel:Beautiful|hotel:comfortable|Rooms:comfortable|Rooms:clean|Rooms:nonoisy|Wifi:excellent|beach:near|mall:near
money:None|hotel:None|hotel:None|hotel:None|Rooms:None|Rooms:None|Rooms:None|Wifi:None|beach:None|mall:None
money:None|hotel:None|hotel:None|Rooms:None|Rooms:None|Rooms:None|Wifi:None|beach:None|mall:None
bed|shower|garden|Room|notowel|pool|breakfast
Room|bed|shower|garden|pool|breakfast
bed:POS|shower:NEU|garden:POS|Room:POS|notowel:NEU|pool:POS|breakfast:POS
bed:POS|shower:POS|garden:POS|Room:POS|notowel:NEG|pool:POS|breakfast:POS
bed:comfortable|shower:hot|garden:lush|Room:big|notowel:Ok|pool:130

In [246]:
metrics

{'accuracy': 0.37625,
 'precision': 0.2317167051578137,
 'recall': 0.2317167051578137,
 'f1': 0.2317167051578137}